# [Chefkoch.de](http://www.chefkoch.de/)
------

### Target:
### Scraping the main recipe collection of over 300,000 different recipes (part 1)



In [1]:
import os
import time
import datetime as dt
from datetime import datetime
from multiprocessing import Pool

from time import sleep, time
from random import randint, choice
import requests
import urllib.request
from bs4 import BeautifulSoup
import csv

Data such as recipe name, rating, date of uploading the recipe, etc. are saved in a csv file.
If the recipe has a picture, the thumbnail is stored in the folder ** search_thumbnails **.

In [2]:
# OS
NOW           = dt.datetime.now()
FILE_NAME     = 'chefkoch' +  '.csv'
DATASET_FOLDER = './input/csv_files/'
IMGS_FOLDER  = './input/images/search_thumbnails/'

# Chefkoch.de Seite
CHEFKOCH_URL  = 'http://www.chefkoch.de'
START_URL     = 'http://www.chefkoch.de/rs/s'
CATEGORY      = '/Rezepte.html'

Alle 300k Rezepte sortiert nach Datum: http://www.chefkoch.de/rs/s30o3/Rezepte.html

Wenn man Website Scrapping durchführt, ist es wichtig die robots.txt Datei zu respektieren. Manche Administratoren möchten nicht das bestimmte Directories von Bots besucht werden. https://www.chefkoch.de/robots.txt liefert:

- User-agent: *  # directed to all spiders, not just Scooter
- Disallow: /cgi-bin
- Disallow: /stats
- Disallow: /pictures/fotoalben/
- Disallow: /forumuploads/
- Disallow: /pictures/user/
- Disallow: /user/
- Disallow: /avatar/
- Disallow: /cms/
- Disallow: /produkte/
- Disallow: /how2videos/

Aufgeführt sind Directories die uns gar nicht interessieren, weshalb man getrost weiter machen kann. Nichtsdestotrotz  sind Massnahmen, wie zufällige Headers und genügend grosse Pausen zwischen den einzelnen Requests empfehlenswert, um einen möglichen Ban der Website zu vermeiden.

In [3]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [4]:
category_url = START_URL + '0o3' + CATEGORY
category_url

'http://www.chefkoch.de/rs/s0o3/Rezepte.html'

In [5]:
category_url = START_URL + '0o3' + CATEGORY

def _get_html(url):
    
    page = ''
    while page == '':
        try:
            page = requests.get(url, headers=random_headers())
        except:
            print('Connection refused')
            time.sleep(10)
            continue
    return page.text

def _get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    #total_pages = soup.find('ul', class_='ds-pagination').find('a', class_='ds-page-link bi-paging-next').text
    total_pages=10
    
    return int(total_pages)



In [6]:
html_text_total_pages = _get_html(category_url)



In [7]:
html_text_total_pages

'<!doctype html>\n<html lang="de" [class]="rahmen_provideDynamicClasses()">\n    <head>\n                \n                    <link rel="preload" as="script" href="https://cdn.ampproject.org/v0.js">\n                <link rel="preconnect dns-prefetch" href="https://fonts.gstatic.com/" crossorigin>\n\n        <meta charset="utf-8">\n\n                    <title>Alle Rezepte | Chefkoch</title>\n            <meta name="description" content="Wir haben 345.868 schmackhafte Rezepte für dich gefunden! Finde was du suchst - lecker &amp; einfach. Jetzt ausprobieren mit ♥ Chefkoch.de ♥.">\n    <meta property="og:title" content="Alle Rezepte | Chefkoch" />\n    <meta property="og:description" content="Wir haben 345.868 schmackhafte Rezepte für dich gefunden! Finde was du suchst - lecker &amp; einfach. Jetzt ausprobieren mit ♥ Chefkoch.de ♥." />\n                <meta property="og:image" content="https://img.chefkoch-cdn.de/rezepte/3846311585928984/bilder/1290338/fit-960x720/getrocknete-tomaten.j

In [8]:
total_pages = _get_total_pages(html_text_total_pages)
print('Total pages: ', total_pages)

Total pages:  10


Liste von allen einzelnen Rezepteurls bei Chefkoch im folgenden Format:
1. Seite: http&#58;//www.chefkoch.de/rs/s**0**o3/Rezepte.html
2. Seite: http&#58;//www.chefkoch.de/rs/s**30**o3/Rezepte.html
3. Seite: http&#58;//www.chefkoch.de/rs/s**60**o3/Rezepte.html
4. Seite: ...

Auf einer Seite erhält man 30 Rezepte. Um jede Seite aufrufen zu können, muss man nur die Zahl zwischen **s** und **o3** um 30 erhöhen.

In [9]:
url_list = []

for i in range(0, total_pages + 1):
    url_to_scrap = START_URL + str(i * 30) + 'o3' + CATEGORY
    print(url_to_scrap)
    url_list.append(url_to_scrap)

http://www.chefkoch.de/rs/s0o3/Rezepte.html
http://www.chefkoch.de/rs/s30o3/Rezepte.html
http://www.chefkoch.de/rs/s60o3/Rezepte.html
http://www.chefkoch.de/rs/s90o3/Rezepte.html
http://www.chefkoch.de/rs/s120o3/Rezepte.html
http://www.chefkoch.de/rs/s150o3/Rezepte.html
http://www.chefkoch.de/rs/s180o3/Rezepte.html
http://www.chefkoch.de/rs/s210o3/Rezepte.html
http://www.chefkoch.de/rs/s240o3/Rezepte.html
http://www.chefkoch.de/rs/s270o3/Rezepte.html
http://www.chefkoch.de/rs/s300o3/Rezepte.html


In [10]:
url_list

['http://www.chefkoch.de/rs/s0o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s30o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s60o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s90o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s120o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s150o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s180o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s210o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s240o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s270o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s300o3/Rezepte.html']

In [11]:
from pprint import pprint
# Die ersten 30 Seiten:
pprint(url_list[:30])

['http://www.chefkoch.de/rs/s0o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s30o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s60o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s90o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s120o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s150o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s180o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s210o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s240o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s270o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s300o3/Rezepte.html']


In [12]:
def _write_to_recipes(data):
    path = DATASET_FOLDER + FILE_NAME
    print(data)
   
    with open(path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow((data['recipe_id'],
                        data['recipe_name'],
                        data['average_rating'],
                        data['stars_shown'],
                        data['votes'],
                        data['difficulty'],
                        data['preparation_time'],
                        data['date'],
                        data['link'],
                        data['has_picture']))

def _get_picture_link(item):
    item_class = item.find('picture').find('img').get('class')
    if item_class == ['lazyload']:
        img_link = item.find('picture').find('img').get('data-srcset')
    else: 
        img_link = item.find('picture').find('source').get('srcset')
    return(img_link)

def _get_front_page(html):
    #print ('check1')
    soup = BeautifulSoup(html, 'lxml')
    lis = soup.find_all('article', class_="rsel-item ds-grid-float ds-col-12 ds-col-m-8")
   
    for index, li in enumerate(lis):

        # get rezept ID
        try:
            id_r = li.get('id')
        except:
            id_r = ''

        # bild speichern falls eins verügbar
        try: 
            if li.find('picture') is not None:
                img_link = _get_picture_link(li)
                img_name = IMGS_FOLDER + str(id_r) + '.jpg'
                urllib.request.urlretrieve(img_link, img_name)
                has_pic = 'yes'
            else: 
                has_pic = 'no'
        except:
            has_pic = ''

        # link
        try:
            link = CHEFKOCH_URL + li.find('a').get('href')
        except:
            link = ''

        # name des rezeptes
        try:
            name = li.find('div', class_='search-list-item-title').text.strip()
        except:
            name = ''

        # durchschnitts bewertung von nutzern
        try:
            stars = li.find('span', class_='search-list-item-uservotes-stars').get('title')
        except:
            stars = ''

        # anzahl sterne
        try:
            stars_shown = li.find('span', class_='search-list-item-uservotes-stars').get('class')[1]
        except:
            stars_shown = ''

        # anzahl votes
        try:
            votes = li.find('span', class_='search-list-item-uservotes-count').text.strip()
        except:
            votes = ''

        # schwierigkeitsgrad des rezeptes => simpel, normal oder pfiffig
        try:
            difficulty = li.find('span', class_='search-list-item-difficulty').text.strip()

        except:
            difficulty = ''

        # zubereitungs zeit
        try:
            preptime = li.find('span', class_='search-list-item-preptime').text.strip()
        except:
            preptime = ''

        # datum
        try:
            date = li.find('span', class_='search-list-item-activationdate').text.strip()
        except:
            date = ''

        # write dictionary
        data = {'recipe_id' : id_r,
                'recipe_name' : name,
                'average_rating': stars,
                'stars_shown' : stars_shown,
                'votes' : votes,
                'difficulty' : difficulty,
                'preparation_time' : preptime,
                'has_picture' : has_pic,
                'date' : date,
                'link' : link[22:]}
        
        # append file
        print('++++++++++++')
        print(data)
        print('++++++++++++')
        _write_to_recipes(data)
        
def scrap_main(url):
    print('Current url: ', url)
    html = _get_html(url)
    _get_front_page(html)
    #sleep(randint(1, 2))
scrap_main( url_list[10])
    

Current url:  http://www.chefkoch.de/rs/s300o3/Rezepte.html
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '', 'difficulty': '', 'preparation_time': '', 'has_picture': 'no', 'date': '', 'link': 'https://www.chefkoch.de/rezepte/3843011585474466/Lende-mit-gebratenem-Apfel.html'}
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '', 'difficulty': '', 'preparation_time': '', 'has_picture': 'no', 'date': '', 'link': 'https://www.chefkoch.de/rezepte/3843011585474466/Lende-mit-gebratenem-Apfel.html'}
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '', 'difficulty': '', 'preparation_time': '', 'has_picture': 'no', 'date': '', 'link': 'https://www.chefkoch.de/rezepte/3842861585430829/Herzhaftes-Dinkelbrot-mit-Zwiebeln-und-Speck.html'}
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '

In [13]:
url_list[10]

'http://www.chefkoch.de/rs/s300o3/Rezepte.html'

In [14]:
scrap_main( url_list[10])

Current url:  http://www.chefkoch.de/rs/s300o3/Rezepte.html
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '', 'difficulty': '', 'preparation_time': '', 'has_picture': 'no', 'date': '', 'link': 'https://www.chefkoch.de/rezepte/3843011585474466/Lende-mit-gebratenem-Apfel.html'}
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '', 'difficulty': '', 'preparation_time': '', 'has_picture': 'no', 'date': '', 'link': 'https://www.chefkoch.de/rezepte/3843011585474466/Lende-mit-gebratenem-Apfel.html'}
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '', 'difficulty': '', 'preparation_time': '', 'has_picture': 'no', 'date': '', 'link': 'https://www.chefkoch.de/rezepte/3842861585430829/Herzhaftes-Dinkelbrot-mit-Zwiebeln-und-Speck.html'}
++++++++++++
{'recipe_id': None, 'recipe_name': '', 'average_rating': '', 'stars_shown': '', 'votes': '

In [15]:

#start_time = time()
#with Pool(1) as p:
    #p.map(scrap_main, url_list[10])
#print("--- %s seconds ---" % (time() - start_time))

## CSV Datei lesen und korrigieren

Unter input/csv_files/ findet man die erstellte CSV Datei.
Grösse: 62.1 MB

In [16]:
!ls input/csv_files/

chefkoch.csv  chefkoch_rezepte_27-12-2017.csv


In [17]:
import pandas as pd
chef_rezepte = pd.read_csv('input/csv_files/chefkoch.csv', header=None)
chef_rezepte.head()

,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.chefkoch.de/rezepte/38584415874633...,no
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.chefkoch.de/rezepte/38584515874650...,no
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.chefkoch.de/rezepte/38584715874662...,no
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.chefkoch.de/rezepte/38589615875476...,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.chefkoch.de/rezepte/38589715875484...,no


In [18]:
chef_rezepte[[8]][:10] # erste 10 zeilen der 8. spalte

,8
0,https://www.chefkoch.de/rezepte/38584415874633...
1,https://www.chefkoch.de/rezepte/38584515874650...
2,https://www.chefkoch.de/rezepte/38584715874662...
3,https://www.chefkoch.de/rezepte/38589615875476...
4,https://www.chefkoch.de/rezepte/38589715875484...
5,https://www.chefkoch.de/rezepte/38391115849675...
6,https://www.chefkoch.de/rezepte/38602515877390...
7,https://www.chefkoch.de/rezepte/38602115877349...
8,https://www.chefkoch.de/rezepte/38601315877296...
9,https://www.chefkoch.de/rezepte/38600315877196...


Beim Scraping ist ein Fehler unterlaufen. Die Links müssen als https Links und nicht als http Links gespeichert werden.
### Korrektur:

In [19]:
chef_rezepte[8] = chef_rezepte[8].str.replace('http', 'https')

In [20]:
chef_rezepte[[8]][:10]

,8
0,httpss://www.chefkoch.de/rezepte/3858441587463...
1,httpss://www.chefkoch.de/rezepte/3858451587465...
2,httpss://www.chefkoch.de/rezepte/3858471587466...
3,httpss://www.chefkoch.de/rezepte/3858961587547...
4,httpss://www.chefkoch.de/rezepte/3858971587548...
5,httpss://www.chefkoch.de/rezepte/3839111584967...
6,httpss://www.chefkoch.de/rezepte/3860251587739...
7,httpss://www.chefkoch.de/rezepte/3860211587734...
8,httpss://www.chefkoch.de/rezepte/3860131587729...
9,httpss://www.chefkoch.de/rezepte/3860031587719...


In [21]:
chef_rezepte.columns = ['recipe_id', 'recipe_name', 'average_rating',
                        'stars_shown', 'votes', 'difficulty', 'preparation_time',
                        'date', 'link', 'has_picture']

In [22]:
chef_rezepte.head()

,recipe_id,recipe_name,average_rating,stars_shown,votes,difficulty,preparation_time,date,link,has_picture
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpss://www.chefkoch.de/rezepte/3858441587463...,no
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpss://www.chefkoch.de/rezepte/3858451587465...,no
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpss://www.chefkoch.de/rezepte/3858471587466...,no
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpss://www.chefkoch.de/rezepte/3858961587547...,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpss://www.chefkoch.de/rezepte/3858971587548...,no


In [23]:
chef_rezepte.to_csv('input/csv_files/chefkoch_rezepte_27-12-2017.csv')

### Umbennenung der Thumbnails: Hänge ein 0 hintendran -> Thumbnail ist das erste Bild des Rezeptes.
#### 1, 2, 3, ..., n Bilder kommen im Teil 2 dazu.

In [24]:
import os
path = 'input/images/search_thumbnails/'
files = os.listdir(path)
i = 1
for file in files:
    filename, file_extension = os.path.splitext(file)
    os.rename(os.path.join(path, file), os.path.join(path, filename + '-0' + file_extension))
    print('renamed: ', i)
    i = i+1

# Weiter gehts mit Teil 2: 02_rezepte_details.ipynb